## Portada del equipo

**Integrantes:**
- Nombre 1 David Leobardo Pérez Cruz
- Nombre 2 Roberto Jhoshua Alegre ventura
- Nombre 3 Mariano Josué Ahuatzi Pichardo
- Nombre 4 Fabian Herrera Barron


In [2]:
# svm_hinge.py
import pandas as pd
import numpy as np

def hinge_loss(w, b, X, y, C):
    """
    L(w,b) = 0.5||w||^2 + C * sum_i max(0, 1 - y_i (w^T x_i + b))
    Devuelve: L (escalar), dW (como w), dB (escalar)
    """
    X = np.asarray(X, float)           # (N, D)
    y = np.asarray(y, float).reshape(-1)  # (N,)
    w = np.asarray(w, float).reshape(-1)  # (D,)
    b = float(b)

    scores  = X @ w + b                # (N,)
    margins = 1.0 - y * scores         # (N,)
    hinge   = np.maximum(0.0, margins) # (N,)

    L  = 0.5 * w.dot(w) + C * hinge.sum()

    active = (margins > 0).astype(float)   # 1 si contribuye
    coeff  = -y * active                   # d(hinge)/d(score)

    dW = w + C * (X.T @ coeff)
    dB = C * coeff.sum()
    return L, dW, dB




In [7]:
if __name__ == "__main__":

    df = pd.read_csv("fase1-labubus/data/train_linear.csv")

    try:
        data = np.genfromtxt("fase1-labubus/data/train_linear.csv", delimiter=",", dtype=float, skip_header=1)
        if np.isnan(data).any():  # si falló por cabecera rara, reintenta sin skip
            raise ValueError
    except Exception:
        data = np.genfromtxt("train_linear.csv", delimiter=",", dtype=float)

    X = data[:, :-1]
    y = data[:, -1]

    # Parámetros iniciales simples
    w = np.zeros(X.shape[1])
    b = 0.0
    C = 1.0

    # Cálculo de pérdida y gradiente
    L, dW, dB = hinge_loss(w, b, X, y, C)

    print(f"N={len(y)}, D={X.shape[1]}")
    print(f"Pérdida inicial: {L:.6f}")
    print(f"||dW||: {np.linalg.norm(dW):.6f}, dB: {dB:.6f}")

    # Una pasada de descenso de gradiente
    lr = 1e-2
    w -= lr * dW
    b -= lr * dB
    L2, _, _ = hinge_loss(w, b, X, y, C)
    print(f"Pérdida tras 1 paso GD: {L2:.6f}")

N=200, D=2
Pérdida inicial: 200.000000
||dW||: 399.661479, dB: 0.000000
Pérdida tras 1 paso GD: 27.602683


1. El término $0.5 \lVert w \rVert^2$ penaliza pesos grandes, lo que ayuda a mantener un margen amplio y evitar sobreajuste.  
   El parámetro $C$ controla el equilibrio entre el tamaño del margen y los errores de clasificación:  
   - $C$ grande → más castigo a errores (margen pequeño, posible sobreajuste).  
   - $C$ pequeño → menos castigo (margen más grande, modelo más general).

2. La parte hinge contribuye $0$ al costo cuando $y_i (w^\top x_i + b) \ge 1$,  
   es decir, cuando el punto está correctamente clasificado y fuera del margen.